In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

import numpy as np


dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

# #normalization_layer = tf.keras.layers.Rescaling(1./255)

def load_images(datapoint): 
  image = tf.image.resize(datapoint["image"],(128,128))

  mask = tf.image.resize(datapoint["segmentation_mask"],(128,128))

  mask = mask - 1
  
  image = tf.cast(image, tf.float32) / 255.0

  return image , mask


train_images = dataset['train'].map(load_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_images = dataset['test'].map(load_images)

train_images = train_images.batch(16)

test_images = test_images.batch(16)


print(train_images)

In [3]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(128,128,3,))
    ]
)

model.add(tf.keras.layers.Conv2D(5,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 
                                 ))

model.add(tf.keras.layers.MaxPool2D((2,2)))

model.add(tf.keras.layers.Conv2D(10,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))

model.add(tf.keras.layers.Conv2D(10,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))

model.add(tf.keras.layers.MaxPool2D((2,2)))

model.add(tf.keras.layers.Conv2D(20,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))

model.add(tf.keras.layers.Conv2D(20,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))

model.add(tf.keras.layers.UpSampling2D((2,2)))

model.add(tf.keras.layers.Conv2D(10,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))

model.add(tf.keras.layers.Conv2D(10,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))

model.add(tf.keras.layers.UpSampling2D((2,2)))

model.add(tf.keras.layers.Conv2D(5,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))

model.add(tf.keras.layers.Conv2D(3,
                                 (3,3),
                                 padding='same',
                                 activation=tf.keras.activations.relu,
                                 ))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 5)       140       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 5)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 10)        460       
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 10)        910       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 10)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 20)        1

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

model.fit(train_images,epochs=5)

In [6]:
score = model.evaluate(test_images, verbose=0)

In [ ]:
print("Test loss:", score[0])
print("Test accuracy:", score[1])